<a href="https://colab.research.google.com/github/OussamaHaff/machine-learning-upskilling/blob/main/02-llms-from-scratch/01-tokenising-text/simple_text_tokenisation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Stats about text to tokenise

In [3]:
with open("data/the-verdict.txt", "r", encoding="utf-8") as book_file:
    raw_text = book_file.read()

print("Total number of characters:", len(raw_text))

Total number of characters: 20479


In [4]:
print("First 100 characters:\n", raw_text[:99])

First 100 characters:
 I HAD always thought Jack Gisburn rather a cheap genius--though a good fellow enough--so it was no 


### Basic Regex



*   Split test *on* whitespace character (`s` for space)

In [5]:
import re

text = "Hello, world. This, is a test."
result_s_split_only = re.split(r'(\s)', text)
print(result_s_split_only)

['Hello,', ' ', 'world.', ' ', 'This,', ' ', 'is', ' ', 'a', ' ', 'test.']


*   Split on whitespace (s), comma (,) and period (.)

In [6]:
result_punc_chars_split = re.split(r'([,.]|\s)', text)
print(result_punc_chars_split)

['Hello', ',', '', ' ', 'world', '.', '', ' ', 'This', ',', '', ' ', 'is', ' ', 'a', ' ', 'test', '.', '']


*   Strip the whitespace character

In [45]:
result_strip_whitespace = [item for item in result_punc_chars_split if item.strip()]
print(result_strip_whitespace)

['Hello', ',', 'world', '.', 'This', ',', 'is', 'a', 'test', '.']


*    Splitg all punctuation characters

In [53]:
text = "Hello, world. Is this-- a test?"
result_all_punc_chars_split = re.split(r'([.,:;?_!"()\']|--|\s)', text)
print(result_all_punc_chars_split)

result_all_punc_chars_split_cleaned = [item.strip() for item in result_all_punc_chars_split if item.strip()]
print("Cleaned result:\n", result_all_punc_chars_split_cleaned)

['Hello', ',', '', ' ', 'world', '.', '', ' ', 'Is', ' ', 'this', '--', '', ' ', 'a', ' ', 'test', '?', '']
Cleaned result:
 ['Hello', ',', 'world', '.', 'Is', 'this', '--', 'a', 'test', '?']


# Basic tokenisation of sample data

In [54]:
preprocessed = re.split(r'([.,:;?_!"()\']|--|\s)', raw_text)
preprocessed = [item.strip() for item in preprocessed if item.strip()]
print("Number of tokens:", len(preprocessed))
print("First 33 tokens:\n", preprocessed[:33])

Number of tokens: 4690
First 33 tokens:
 ['I', 'HAD', 'always', 'thought', 'Jack', 'Gisburn', 'rather', 'a', 'cheap', 'genius', '--', 'though', 'a', 'good', 'fellow', 'enough', '--', 'so', 'it', 'was', 'no', 'great', 'surprise', 'to', 'me', 'to', 'hear', 'that', ',', 'in', 'the', 'height', 'of']


# Basic vocabulary of sample data

In [55]:
vocab_words = sorted(set(preprocessed))
vocab_size = len(vocab_words)
print("Vocabulary size:", vocab_size)

Vocabulary size: 1130


### The actual vocabulary with IDs

In [56]:
vocab = { token:integer for integer,token in enumerate(vocab_words) }
for i, item in enumerate(vocab.items()):
  print(item)
  if i > 50:
    break

('!', 0)
('"', 1)
("'", 2)
('(', 3)
(')', 4)
(',', 5)
('--', 6)
('.', 7)
(':', 8)
(';', 9)
('?', 10)
('A', 11)
('Ah', 12)
('Among', 13)
('And', 14)
('Are', 15)
('Arrt', 16)
('As', 17)
('At', 18)
('Be', 19)
('Begin', 20)
('Burlington', 21)
('But', 22)
('By', 23)
('Carlo', 24)
('Chicago', 25)
('Claude', 26)
('Come', 27)
('Croft', 28)
('Destroyed', 29)
('Devonshire', 30)
('Don', 31)
('Dubarry', 32)
('Emperors', 33)
('Florence', 34)
('For', 35)
('Gallery', 36)
('Gideon', 37)
('Gisburn', 38)
('Gisburns', 39)
('Grafton', 40)
('Greek', 41)
('Grindle', 42)
('Grindles', 43)
('HAD', 44)
('Had', 45)
('Hang', 46)
('Has', 47)
('He', 48)
('Her', 49)
('Hermia', 50)
('His', 51)


# Simple Text Tokeniser

In [60]:
class SimpleTextTokeniserV1:
  def __init__(self, vocab):
    """
    Saves the vocabulary of unique and sorted tokens,
    then creates its inverse and saves it.
    """
    self.str_to_int = vocab
    self.int_to_str = { i:s for s,i in vocab.items() }

  def encode(self, text):
    """
    Preprocesses a text by splitting it on special chars,
    then strip the text from all those chars.
    then map the tokens to ids based on the vocab
    """
    preprocessed = re.split(r'([.,:;!?()"\']|--|\s)', text)
    preprocessed = [item.strip() for item in preprocessed if item.strip()]
    ids = [self.str_to_int[s] for s in preprocessed]
    return ids

  def decode(self, ids):
    text = " ".join([self.int_to_str[i] for i in ids])
    # Subtitues any occurence of a space + punctuation with the punctuation char
    text = re.sub(r'\s+([,.?!()"\'])', r'\1', text)
    return text


### Applying the tokeniser on the text

#### Encode

In [61]:
tokeniser = SimpleTextTokeniserV1(vocab)
text = """
  "It's the last he painted, you know," Mrs. Gisburn said with pardonable pride.
  """
ids = tokeniser.encode(text)
print(ids)

[1, 56, 2, 850, 988, 602, 533, 746, 5, 1126, 596, 5, 1, 67, 7, 38, 851, 1108, 754, 793, 7]


#### Decode

In [64]:
print(tokeniser.decode(ids))

" It' s the last he painted, you know," Mrs. Gisburn said with pardonable pride.


#### Encode unknown tokens

In [66]:
text = "Hello, do you like tea?"
# This should print an error since 'Hello' does not exist in the original dataset
print(tokeniser.encode(text))

KeyError: 'Hello'

#### Special tokens

In [82]:
all_vocab_tokens = sorted(set(preprocessed))
all_vocab_tokens.extend(["<|endoftext|>", "<|unk|>"])
vocab = { token:integer for integer,token in enumerate(all_vocab_tokens) }

print("Vocab length after adding 2 special chars:", len(vocab.items()))

Vocab length after adding 2 special chars: 1132


In [83]:
last_vocab_tokens = enumerate(list(vocab.items())[-5:])
for i,item in last_vocab_tokens:
  print(item)


('younger', 1127)
('your', 1128)
('yourself', 1129)
('<|endoftext|>', 1130)
('<|unk|>', 1131)


In [84]:
class SimpleTextTokeniserV2:
  def __init__(self, vocab):
    """
    Saves the vocabulary of unique and sorted tokens,
    then creates its inverse and saves it.
    """
    self.str_to_int = vocab
    self.int_to_str = { i:s for s,i in vocab.items() }

  def encode(self, text):
    """
    Preprocesses a text by splitting it on special chars,
    then strip the text from all those chars.
    then map the tokens to ids based on the vocab
    """
    preprocessed = re.split(r'([.,:;!?()"\']|--|\s)', text)
    preprocessed = [item.strip() for item in preprocessed if item.strip()]
    preprocessed = [
        item if item in self.str_to_int else "<|unk|>" for item in preprocessed
        ]
    ids = [self.str_to_int[s] for s in preprocessed]
    return ids

  def decode(self, ids):
    text = " ".join([self.int_to_str[i] for i in ids])
    # Subtitues any occurence of a space + punctuation with the punctuation char
    text = re.sub(r'\s+([,.?!()"\'])', r'\1', text)
    return text

#### Encode with special tokens

In [89]:
text1 = "Hello, do you like tea?"
text2 = "In the sunlit terrace of the palace."
text = " <|endoftext|> ".join((text1, text2))
print(text)

Hello, do you like tea? <|endoftext|> In the sunlit terrace of the palace.


In [90]:
tokeniserV2 = SimpleTextTokeniserV2(vocab)
encoded_text = tokeniserV2.encode(text)
print(encoded_text)

[1131, 5, 355, 1126, 628, 975, 10, 1130, 55, 988, 956, 983, 722, 988, 1131, 7]


#### Decode with special tokens

In [91]:
tokeniserV2.decode(encoded_text)

'<|unk|>, do you like tea? <|endoftext|> In the sunlit terrace of the <|unk|>.'